In [9]:
import random
import pandas as pd

# Générer des séquences aléatoires
items = ["Telephone", "Ordinateur", "Ecouteurs", "Batterie", "Cle-USB", "Souris", "Clavier", "Sac", "Disque-dur"]
num_sequences = 20
max_length = 7

sequences = []
for _ in range(num_sequences):
    length = random.randint(3, max_length)
    sequence = [(item, tid + 1) for tid, item in enumerate(random.sample(items, length))]
    sequences.append(sequence)

# Convertir les séquences en DataFrame
df = pd.DataFrame([{"Sequence": i + 1, "Item": item, "ID": tid} for i, seq in enumerate(sequences) for item, tid in seq])

# Afficher le DataFrame
print("DataFrame:")
df.head(50)

DataFrame:


,Sequence,Item,ID
0,1,Souris,1
1,1,Clavier,2
2,1,Sac,3
3,2,Cle-USB,1
4,2,Ordinateur,2
5,2,Ecouteurs,3
6,3,Souris,1
7,3,Disque-dur,2
8,3,Ecouteurs,3
9,3,Telephone,4


In [10]:


# Convertir le DataFrame en dictionnaire de séquences
sequences_dict = defaultdict(list)
for _, row in df.iterrows():
    sequences_dict[row['Sequence']].append((row['Item'], row['ID']))

# Fonction pour vérifier si un motif est une sous-séquence d'une transaction
# Fonction pour vérifier si un motif est une sous-séquence d'une transaction
# Fonction pour vérifier si un motif est une sous-séquence d'une transaction
def is_subsequence(subseq, seq):
    it = iter(seq)
    for item in subseq:
        found = False
        for trans_item in it:
            trans_item_set = set([trans_item[0]])  # Prendre uniquement l'article et non l'ID
            # Vérifiez si item est une séquence ou un élément unique
            if isinstance(item, tuple) or isinstance(item, list):
                # Si item est un tuple ou une liste, comparez comme un ensemble
                if set(item).issubset(trans_item_set):
                    found = True
                    break
            else:
                # Si item est un élément unique, comparez directement
                if {item}.issubset(trans_item_set):
                    found = True
                    break
        if not found:
            return False
    return True


# Générer les motifs fréquents d'un seul élément
def get_single_item_frequent_patterns(sequences, min_support):
    item_support = defaultdict(int)
    
    for sid, sequence in sequences.items():
        unique_items_in_seq = set()
        for itemset in sequence:
            for item in itemset:
                unique_items_in_seq.add(item)
        
        for item in unique_items_in_seq:
            item_support[(item,)] += 1
    
    frequent_patterns = {pattern: support for pattern, support in item_support.items() if support >= min_support}
    
    return frequent_patterns

# Générer des candidats à partir de motifs fréquents précédents
def generate_candidates(prev_frequent_patterns):
    candidates = set()
    patterns = list(prev_frequent_patterns.keys())
    
    for i in range(len(patterns)):
        for j in range(i, len(patterns)):
            p1, p2 = patterns[i], patterns[j]
            # Combiner les motifs fréquents pour créer des motifs séquentiels plus longs
            new_pattern = p1 + p2
            candidates.add(new_pattern)
    
    return candidates

# Calculer le support des candidats
def count_support(sequences, candidates):
    support_count = defaultdict(int)
    
    for candidate in candidates:
        for sid, sequence in sequences.items():
            if is_subsequence(candidate, sequence):
                support_count[candidate] += 1
    
    return support_count

# SPADE Algorithm
def spade_algorithm(sequences, min_support):
    # Obtenir les motifs fréquents d'un seul élément
    frequent_patterns = get_single_item_frequent_patterns(sequences, min_support)
    all_frequent_patterns = frequent_patterns.copy()
    
    # Étendre les motifs d'un seul élément en motifs plus longs
    k = 2
    while True:
        # Générer les candidats à partir des motifs fréquents de la précédente étape
        candidates = generate_candidates(frequent_patterns)
        
        # Compter le support des candidats générés
        candidate_support = count_support(sequences, candidates)
        
        # Filtrer les motifs fréquents
        frequent_patterns = {pattern: support for pattern in candidate_support.items() if support >= min_support}
        
        if not frequent_patterns:
            break  # Arrêter si aucun nouveau motif fréquent n'est trouvé
        
        # Ajouter les nouveaux motifs fréquents aux résultats finaux
        all_frequent_patterns.update(frequent_patterns)
        k += 1
    
    return all_frequent_patterns

# Minimum support
min_support = 2

# Appliquer l'algorithme SPADE
frequent_patterns = spade_algorithm(sequences_dict, min_support)

# Afficher les motifs séquentiels fréquents
print("Motifs séquentiels fréquents trouvés par SPADE :")
for pattern, support in frequent_patterns.items():
    print(f"Motif : {pattern}, Support : {support}")


Motifs séquentiels fréquents trouvés par SPADE :
Motif : (1,), Support : 20
Motif : (2,), Support : 20
Motif : (3,), Support : 20
Motif : ('Souris',), Support : 9
Motif : ('Clavier',), Support : 11
Motif : ('Sac',), Support : 13
Motif : ('Ordinateur',), Support : 13
Motif : ('Ecouteurs',), Support : 7
Motif : ('Cle-USB',), Support : 11
Motif : (4,), Support : 13
Motif : (5,), Support : 10
Motif : ('Telephone',), Support : 12
Motif : ('Disque-dur',), Support : 11
Motif : ('Batterie',), Support : 8
Motif : (6,), Support : 7
Motif : (7,), Support : 5
Motif : (('Clavier', 'Cle-USB'), 2), Support : 3
Motif : (('Clavier', 'Disque-dur'), 2), Support : 3
Motif : (('Ecouteurs', 'Batterie'), 1), Support : 3
Motif : (('Cle-USB', 'Batterie'), 2), Support : 3
Motif : (('Cle-USB', 'Telephone'), 3), Support : 3
Motif : (('Souris', 'Sac'), 4), Support : 3
Motif : (('Sac', 'Cle-USB'), 2), Support : 3
Motif : (('Sac', 'Disque-dur'), 3), Support : 3
Motif : (('Ecouteurs', 'Disque-dur'), 1), Support : 3
M